In [1]:
!pip install sagemaker --upgrade


/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 438 kB 26.0 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.54.0-py2.py3-none-any.whl size=614442 sha256=7b54bbad9b17b8616ce2d730d8af15b934fbc619f23ad244d90d6a954130386e
  Stored in directory: /root/.cache/pip/wheels/9a/af/74/da0474859873c1cb39ae17013579ec8dcbabae7cc8cb7a7dbd
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.49.1
    Uninstalling sagemaker-2.49.1:
      Successfully uninstalled sagemaker-2.49.1
You should consider upgrading via the '/opt/conda/bin/pyth

https://towardsdatascience.com/huggingface-processing-jobs-on-amazon-sagemaker-b1f5af97b663

In [3]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='dsoaws/1.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region,
                            config=config)
s3 = boto3.Session().client(service_name="s3", 
                            region_name=region,
                            config=config)

## 1. Download Dataset to Local

https://s3.amazonaws.com/amazon-reviews-pds/readme.html

In [4]:
!aws s3 ls s3://amazon-reviews-pds/tsv/


2017-11-24 13:22:50          0 
2017-11-24 13:48:03  241896005 amazon_reviews_multilingual_DE_v1_00.tsv.gz
2017-11-24 13:48:17   70583516 amazon_reviews_multilingual_FR_v1_00.tsv.gz
2017-11-24 13:48:34   94688992 amazon_reviews_multilingual_JP_v1_00.tsv.gz
2017-11-24 13:49:14  349370868 amazon_reviews_multilingual_UK_v1_00.tsv.gz
2017-11-24 13:48:47 1466965039 amazon_reviews_multilingual_US_v1_00.tsv.gz
2017-11-24 13:49:53  648641286 amazon_reviews_us_Apparel_v1_00.tsv.gz
2017-11-24 13:56:36  582145299 amazon_reviews_us_Automotive_v1_00.tsv.gz
2017-11-24 14:04:02  357392893 amazon_reviews_us_Baby_v1_00.tsv.gz
2017-11-24 14:08:11  914070021 amazon_reviews_us_Beauty_v1_00.tsv.gz
2017-11-24 14:17:41 2740337188 amazon_reviews_us_Books_v1_00.tsv.gz
2017-11-24 14:45:50 2692708591 amazon_reviews_us_Books_v1_01.tsv.gz
2017-11-24 15:10:21 1329539135 amazon_reviews_us_Books_v1_02.tsv.gz
2017-11-24 15:22:13  442653086 amazon_reviews_us_Camera_v1_00.tsv.gz
2017-11-24 15:27:13 2689739299 amazon_rev

In [21]:

!aws s3 cp  s3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz .
!aws s3 cp  s3://amazon-reviews-pds/tsv/sample_us.tsv .

download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz to ./amazon_reviews_us_Digital_Software_v1_00.tsv.gz
download: s3://amazon-reviews-pds/tsv/sample_us.tsv to ./sample_us.tsv


## 2. Data Visualization

In [7]:

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

%matplotlib inline

In [8]:
#df = pd.read_csv('./amazon_reviews_us_Music_v1_00.tsv.gz', compression='gzip',sep='\t', nrows=100)
df = pd.read_csv('./sample_us.tsv',sep='\t')

In [9]:
df.head()


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,18778586,RDIJS7QYB6XNR,B00EDBY7X8,122952789,Monopoly Junior Board Game,Toys,5,0,0,N,Y,Five Stars,Excellent!!!,2015-08-31
1,US,24769659,R36ED1U38IELG8,B00D7JFOPC,952062646,56 Pieces of Wooden Train Track Compatible wit...,Toys,5,0,0,N,Y,Good quality track at excellent price,Great quality wooden track (better than some o...,2015-08-31
2,US,44331596,R1UE3RPRGCOLD,B002LHA74O,818126353,Super Jumbo Playing Cards by S&S Worldwide,Toys,2,1,1,N,Y,Two Stars,Cards are not as big as pictured.,2015-08-31
3,US,23310293,R298788GS6I901,B00ARPLCGY,261944918,Barbie Doll and Fashions Barbie Gift Set,Toys,5,0,0,N,Y,my daughter loved it and i liked the price and...,my daughter loved it and i liked the price and...,2015-08-31
4,US,38745832,RNX4EXOBBPN5,B00UZOPOFW,717410439,Emazing Lights eLite Flow Glow Sticks - Spinni...,Toys,1,1,1,N,Y,DONT BUY THESE!,Do not buy these! They break very fast I spun ...,2015-08-31


## 3. SageMaker Processing Job

### 3.1 Copy Data From the Public S3 Bucket to our Private S3 Bucket in this Account

In [48]:
s3_public_path_tsv = "s3://amazon-reviews-pds/tsv"
%store s3_public_path_tsv

s3_private_path_tsv = "s3://{}/amazon-reviews-pds/tsv".format(bucket)
print(s3_private_path_tsv)
%store s3_private_path_tsv

Stored 's3_public_path_tsv' (str)
s3://sagemaker-us-east-1-869312567674/amazon-reviews-pds/tsv
Stored 's3_private_path_tsv' (str)


In [49]:
!aws s3 cp --recursive $s3_public_path_tsv/ $s3_private_path_tsv/ --exclude "*" --include "amazon_reviews_us_Digital_Software_v1_00.tsv.gz"

copy: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz to s3://sagemaker-us-east-1-869312567674/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz


In [50]:
filepath = "{}/amazon_reviews_us_Digital_Software_v1_00.tsv.gz".format(s3_private_path_tsv)
print(filepath)

s3://sagemaker-us-east-1-869312567674/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz


### 3.2 Setup Processing Job

In [51]:
from sagemaker.huggingface.processing import HuggingFaceProcessor

hf_processor = HuggingFaceProcessor(
    role=role,
    instance_type="ml.p3.2xlarge",
    transformers_version='4.6',
    pytorch_version='1.7',
    instance_count=1,
)

In [54]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
inputs = [ProcessingInput(source=filepath, destination="/opt/ml/processing/input")]
outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/training"),
        ProcessingOutput(output_name="validation_data", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/test"),
    ]
arguments = ["--file-name", "amazon_reviews_us_Digital_Software_v1_00.tsv.gz",
             "--model-name", "distilbert-base-uncased",
             "--train-ratio", "0.7",
             "--val-ratio", "0.15",
             "--star-threshold", "4"]

In [55]:
hf_processor.run(
    code="./preprocessing-hf.py",
    inputs=inputs,
    outputs=outputs,
    arguments=arguments
)


Job Name:  huggingface-2021-08-24-15-00-53-465
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-869312567674/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-869312567674/huggingface-2021-08-24-15-00-53-465/source/sourcedir.tar.gz', 'LocalPath': '/opt/ml/processing/input/code/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'entrypoint', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-869312567674/huggingface-2021-08-24-15-00-53-465/source/runproc.sh', 'LocalPath': '/opt/ml/processing/input/entrypoint', 'S3DataType': 'S3Prefix', 'S3InputMode': 'F

In [57]:
preprocessing_job_description = hf_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    print(output['S3Output']['S3Uri'])

s3://sagemaker-us-east-1-869312567674/huggingface-2021-08-24-15-00-53-465/output/train_data
s3://sagemaker-us-east-1-869312567674/huggingface-2021-08-24-15-00-53-465/output/validation_data
s3://sagemaker-us-east-1-869312567674/huggingface-2021-08-24-15-00-53-465/output/test_data
